In [2]:
import pandas as pd

import os
import time

import warnings
warnings.filterwarnings('ignore')

In [3]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys


from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
option = webdriver.ChromeOptions()

# this is the preference we 're passing
# Pass the argument 1 to allow and 2 to block
# cambiamos el directorio por defecto para las descargas 
# (aquí hay que poner la ruta para las descargas por defecto, 
# aunque en nuestra función vamos a sustituir esta ruta por la iteración)
prefs = {
   'profile.default_content_setting_values.automatic_downloads': 1, 
   "profile.default_content_setting_values.notifications": 2, 
   "download.default_directory" : "ruta_descargas"
    }

option.add_experimental_option("prefs", prefs)

#enabled/disabled cookies/notifications/click accept.
option.add_experimental_option("excludeSwitches", ["enable-automation"])
option.add_experimental_option('useAutomationExtension', False)
option.add_argument("--disable-infobars")
option.add_argument("start-maximized")
option.add_argument("--disable-extensions")

#chrome to stay open
option.add_experimental_option("detach", True)

In [28]:
# abrir csv
df = pd.read_csv("ruta_csv")

In [29]:
# crear la lista con todas las urls que queremos descargar, lo hacemos en forma de lista.
# usaremos esta variable cómo parámetro para nuestra función descargar
links = list(df["Link"].values)

In [30]:
# creamos el directorio dónde vamos a descargar los ficheros.
os.mkdir("ruta_directorio_nuevo")

In [31]:
# creamos la variable ruta, path del nuevo directorio, para usarla cómo parámetro de nuestra función descargar.
ruta = "ruta_directorio_nuevo"

In [32]:
# función para esperar a que terminen las descargas antes de pasar a la siguiente instrucción
# al llamarla tenemos que pasarle la variable ruta cómo parámetro

def download_wait(path_to_downloads):
    seconds = 0
    dl_wait = True
    while dl_wait and seconds < 250:
        time.sleep(1)
        dl_wait = False
        for fname in os.listdir(path_to_downloads):
            if fname.endswith('.crdownload'):
                dl_wait = True
        seconds += 1
    return seconds

In [33]:
# voy a componer la función descargar pero creando directorios para cada una de los rows, 
# el título de las grabaciones será el nombre del directorio contenedor de los documentos

def descargar (url, ruta):

    for i in url:
        
        indice = df.loc[df["Link"] == i].index
        nombre_directorio = df["Título"][indice[0]]

        os.chdir(ruta)
        os.mkdir(nombre_directorio)

        dir_descarga =f"{ruta}/{nombre_directorio}"

        print(dir_descarga)
        # cambiamos el directorio por defecto para las descargas
        prefs = {
                'profile.default_content_setting_values.automatic_downloads': 1, "profile.default_content_setting_values.notifications": 2, "download.default_directory" : dir_descarga
                }

        option.add_experimental_option("prefs", prefs)

        driver = webdriver.Chrome(executable_path="/usr/bin/chromedriver",options=option)

        lista_nombre_url = i.split("/")
        if lista_nombre_url.count("webhook_download") == 1:
            driver.get(i)

            download_wait(dir_descarga)
            print("Descarga webhook completada")

            #cierro el driver antes de salir
            driver.quit()
            
        else:

            driver.get(i)
            driver.find_element(By.CSS_SELECTOR, ".download-btn").click()
    
            download_wait(dir_descarga)
            print("Descarga CSS1 completada")

            if len(os.listdir(dir_descarga)) == 0:
                print("Voy a intentar con el otro CSS")
                
                driver.find_element(By.CSS_SELECTOR, ".zm-icon-download").click()
    
                download_wait(dir_descarga)
                print("Descarga CSS2 completada")

                #cierro el driver antes de salir
                driver.quit()
            driver.quit()

In [34]:
# ejecutamos la función: tenemos que meterle la lista de links(df["Link"].values) y la ruta dónde queremos guardar los directorios
descargar(links, ruta)

/home/cachito/Adalab/grabaciones/modulo2/Introducción  a Numpy
Descarga CSS1 completada
/home/cachito/Adalab/grabaciones/modulo2/Introducción  a Pandas Series y Dataframes
Descarga CSS1 completada
/home/cachito/Adalab/grabaciones/modulo2/Presentación Proyecto 2
Descarga CSS1 completada
/home/cachito/Adalab/grabaciones/modulo2/Numpy 2 Indexación en Numpy
Descarga CSS1 completada
/home/cachito/Adalab/grabaciones/modulo2/Pandas 2 Carga y guardado de datos
Descarga CSS1 completada
/home/cachito/Adalab/grabaciones/modulo2/Proyecto: Crear tablero kanban en projectos Adalab
Descarga CSS1 completada
/home/cachito/Adalab/grabaciones/modulo2/Numpy III - Operaciones estadísticas y matemáticas

Descarga CSS1 completada
/home/cachito/Adalab/grabaciones/modulo2/Pandas III - Métodos Pandas
Descarga CSS1 completada
/home/cachito/Adalab/grabaciones/modulo2/NumPy IV - Conjuntos, distribuciones aleatorias
Descarga CSS1 completada
/home/cachito/Adalab/grabaciones/modulo2/Pandas IV - Filtrado de datos
Desc

In [35]:
# comprobamos que nos ha generado todos los directorios

len(os.listdir(ruta)) == len(df["Link"])

True